In [ ]:
# import libraries
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
from scipy.sparse import dia_matrix
#from sympy import *

In [ ]:
# set parameters & initialize grids (reach 1 chosen)
s0 = 0.0005
L = 74000 #m
width = 100 #m
t_tot = 36*60*60 #36 hrs

# initialize grids
dx = 250 #m
dt = 60 #s

# total time step and grid points we're running
m = int(L/dx) # number of spatial points, m
n = int(t_tot/dt) # number of time steps, n

# diffusivity coef
D = 2325 #m^2/s
# celerity
Ce = 1.19 #m/s

# Ca
Ca = Ce*(dt/dx)
# Cd
Cd = D*(dt/(dx**2))

In [ ]:
# pre-allocate vectors 
Qi_k = np.zeros([m,1]) # all spatial solutions at time k (starting from initial)

Qall = np.zeros([m,n]) # collection of Q vectors, compilation of solutions at all timesteps

Ls = np.linspace(0,L,m) 
ts = np.linspace(0,t_tot,n) 

# print(Ls)
# print(ts)

In [ ]:
# Account for boundary conditions?
Qi_k[0] = Q*(Ca/4 - Cd/2) + Q*(1+Cd) - Q(Ca/4 + Cd/2)

Qall[:,0] = Qi_k.transpose()

In [ ]:
# Create diffusion evolution matrix
# M1, M2
# approach: make a sparse matrix with diagonal elements
M1 = dia_matrix((m,m),dtype=np.double).toarray()
M2 = dia_matrix((m,m),dtype=np.double).toarray()

# 1.populate M1
# we want 1+Cd on the diagonal, (Ca/4 - Cd/2) along right subdiagonal, & -(Ca/4 + Cd/2) along left subdiagonal
for i in np.arange(0,m):
    for j in np.arange(0,m):
        if i==j: # we're on the diagonal
            M1[i,j] = 1+Cd
        elif i-1==j: # we're on the left sub-diagonal
            M1[i,j] = Ca/4 - Cd/2
        elif i+1==j: # we're on the right sub-diagonal
            M1[i,j] = -(Ca/4 + Cd/2)
print(M1)

# 2.populate M2
# we want 1-Cd on the diagonal, -(Ca/4 - Cd/2) along right subdiagonal, & (Ca/4 + Cd/2) along left subdiagonal
for i in np.arange(0,m):
    for j in np.arange(0,m):
        if i==j: # we're on the diagonal
            M2[i,j] = 1-Cd
        elif i-1==j: # we're on the left sub-diagonal
            M2[i,j] = -(Ca/4 - Cd/2)
        elif i+1==j: # we're on the right sub-diagonal
            M2[i,j] = (Ca/4 + Cd/2)
print(M2)

In [ ]:
# run model loop
# update Qi_k at each time step

for t in np.arange(0,n): # for every index going from 0 to the total number of timesteps
    time = ts[t] # time is the corrsponding element in the ts vector
    # surface t
    Qi_k[0] = Q*(Ca/4 - Cd/2) + Q*(1+Cd) - Q(Ca/4 + Cd/2)
    # deepest t
    
    #Qi_k[?] = ??
    
    # run matrix solve
    M1_inv = np.linalg.inv(M1)
    Qnew_pre = np.matmul(M1_inv,M2)
    Qnew = np.matmul(Qnew_pre,Qi_k)
    
    #Qnew[?]=??
    Qall[:,t] = Qnew.transpose()
    Qi_k = Qnew
    #print(Qi_k)